In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import sys
import itertools
import random
import torch
import os
import signal
import gc
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Conv2DTranspose, Activation, MaxPool2D, UpSampling2D,Reshape, Flatten, Dense, Dropout,GlobalAveragePooling2D
from keras import backend as K
from keras.utils import plot_model
from numba import cuda,jit
import multiprocessing as mp
from multiprocessing import Pool, cpu_count
import psutil

np.set_printoptions(threshold=sys.maxsize,linewidth=512)
#np.set_printoptions(threshold=256)

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))
 
#여백 줄이기
display(HTML("<style>.prompt { min-width: 30ex !important; }</style>"))

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

In [ ]:
##################################################################################################################

In [4]:
#np.save("./dataset2/1000/input100k.npy",r_input)
#np.save("./dataset2/1000/output100k.npy",r_output)

r_input=np.load("./dataset3/input500k.npy")
r_output=np.load("./dataset3/output500k_num_class.npy")



print(r_input.shape)
print(r_output.shape)
print(test_input.shape)
print(test_output.shape)



(500000, 32, 64, 3)
(500000, 10)
(1000, 32, 64, 3)
(1000, 10)


In [5]:
#resnet50 = tf.keras.applications.resnet.ResNet50(weights='imagenet', input_shape = (128,128,3),include_top=False)
#resnet50.summary()

eff_net=tf.keras.applications.EfficientNetV2L(weights="imagenet",input_shape=(32,64,3),include_top=False)
eff_net.summary()


473176280/473176280 [==============================] - 20s 0us/step
Model: "efficientnetv2-l"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 64, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 64, 3)    0           ['input_1[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 16, 32, 32)   864         ['rescaling[0][0]']              
                                                                                                  
 stem_bn (BatchNormalization)   (None, 16, 32, 32)   128         ['stem_conv[0][0]']              
               

 block2b_expand_conv (Conv2D)   (None, 8, 16, 256)   147456      ['block2a_project_bn[0][0]']     
                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 8, 16, 256)  1024        ['block2b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block2b_expand_activation (Act  (None, 8, 16, 256)  0           ['block2b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block2b_project_conv (Conv2D)  (None, 8, 16, 64)    16384       ['block2b_expand_activation[0][0]
                                                                 ']                               
          

                                                                                                  
 block2f_project_conv (Conv2D)  (None, 8, 16, 64)    16384       ['block2f_expand_activation[0][0]
                                                                 ']                               
                                                                                                  
 block2f_project_bn (BatchNorma  (None, 8, 16, 64)   256         ['block2f_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block2f_drop (Dropout)         (None, 8, 16, 64)    0           ['block2f_project_bn[0][0]']     
                                                                                                  
 block2f_add (Add)              (None, 8, 16, 64)    0           ['block2f_drop[0][0]',           
          

 block3d_expand_conv (Conv2D)   (None, 4, 8, 384)    331776      ['block3c_add[0][0]']            
                                                                                                  
 block3d_expand_bn (BatchNormal  (None, 4, 8, 384)   1536        ['block3d_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block3d_expand_activation (Act  (None, 4, 8, 384)   0           ['block3d_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block3d_project_conv (Conv2D)  (None, 4, 8, 96)     36864       ['block3d_expand_activation[0][0]
                                                                 ']                               
          

                                                                                                  
 block4a_dwconv2 (DepthwiseConv  (None, 2, 4, 384)   3456        ['block4a_expand_activation[0][0]
 2D)                                                             ']                               
                                                                                                  
 block4a_bn (BatchNormalization  (None, 2, 4, 384)   1536        ['block4a_dwconv2[0][0]']        
 )                                                                                                
                                                                                                  
 block4a_activation (Activation  (None, 2, 4, 384)   0           ['block4a_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block4a_s

...

                                                                                                  
 block6w_se_excite (Multiply)   (None, 1, 2, 2304)   0           ['block6w_activation[0][0]',     
                                                                  'block6w_se_expand[0][0]']      
                                                                                                  
 block6w_project_conv (Conv2D)  (None, 1, 2, 384)    884736      ['block6w_se_excite[0][0]']      
                                                                                                  
 block6w_project_bn (BatchNorma  (None, 1, 2, 384)   1536        ['block6w_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6w_drop (Dropout)         (None, 1, 2, 384)    0           ['block6w_project_bn[0][0]']     
          

                                                                                                  
 block6y_project_bn (BatchNorma  (None, 1, 2, 384)   1536        ['block6y_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6y_drop (Dropout)         (None, 1, 2, 384)    0           ['block6y_project_bn[0][0]']     
                                                                                                  
 block6y_add (Add)              (None, 1, 2, 384)    0           ['block6y_drop[0][0]',           
                                                                  'block6x_add[0][0]']            
                                                                                                  
 block7a_expand_conv (Conv2D)   (None, 1, 2, 2304)   884736      ['block6y_add[0][0]']            
          

                                                                                                  
 block7c_expand_bn (BatchNormal  (None, 1, 2, 3840)  15360       ['block7c_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block7c_expand_activation (Act  (None, 1, 2, 3840)  0           ['block7c_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block7c_dwconv2 (DepthwiseConv  (None, 1, 2, 3840)  34560       ['block7c_expand_activation[0][0]
 2D)                                                             ']                               
                                                                                                  
 block7c_b

 ivation)                                                                                         
                                                                                                  
 block7e_dwconv2 (DepthwiseConv  (None, 1, 2, 3840)  34560       ['block7e_expand_activation[0][0]
 2D)                                                             ']                               
                                                                                                  
 block7e_bn (BatchNormalization  (None, 1, 2, 3840)  15360       ['block7e_dwconv2[0][0]']        
 )                                                                                                
                                                                                                  
 block7e_activation (Activation  (None, 1, 2, 3840)  0           ['block7e_bn[0][0]']             
 )                                                                                                
          

 block7g_bn (BatchNormalization  (None, 1, 2, 3840)  15360       ['block7g_dwconv2[0][0]']        
 )                                                                                                
                                                                                                  
 block7g_activation (Activation  (None, 1, 2, 3840)  0           ['block7g_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block7g_se_squeeze (GlobalAver  (None, 3840)        0           ['block7g_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block7g_se_reshape (Reshape)   (None, 1, 1, 3840)   0           ['block7g_se_squeeze[0][0]']     
          

In [6]:

model3 = tf.keras.Sequential([
    eff_net,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(320, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')

])

model3.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model3.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Functiona  (None, 1, 2, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 320)               409920    
                                                                 
 dropout_1 (Dropout)         (None, 320)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                3

In [7]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
save = tf.keras.callbacks.ModelCheckpoint('best_model_effv2L_0413.h5', monitor='val_loss', mode='min', save_best_only=True)

def scheduler(epoch):
    if epoch < 20:
        return 0.001
    else:
        return 0.001*math.exp(0.1*(10-epoch))

schedule = tf.keras.callbacks.LearningRateScheduler(scheduler)


callback = [es, save, schedule]

model3.fit(r_input,r_output, epochs=100, batch_size = 128, validation_split =0.1, callbacks=callback )    
model3.save("./weights_efficientv2.h5")

Epoch 1/100
3516/3516 [==============================] - 409s 109ms/step - loss: 0.3975 - accuracy: 0.8319 - val_loss: 0.3245 - val_accuracy: 0.8639 - lr: 0.0010
Epoch 2/100
3516/3516 [==============================] - 379s 108ms/step - loss: 0.2992 - accuracy: 0.8741 - val_loss: 0.4093 - val_accuracy: 0.8514 - lr: 0.0010
Epoch 3/100
3516/3516 [==============================] - 377s 107ms/step - loss: 0.2596 - accuracy: 0.8900 - val_loss: 18.6839 - val_accuracy: 0.2721 - lr: 0.0010
Epoch 4/100
3516/3516 [==============================] - 379s 108ms/step - loss: 0.2366 - accuracy: 0.8985 - val_loss: 9.0019 - val_accuracy: 0.0174 - lr: 0.0010
Epoch 5/100
3516/3516 [==============================] - 378s 107ms/step - loss: 0.2167 - accuracy: 0.9074 - val_loss: 3.0728 - val_accuracy: 0.3460 - lr: 0.0010
Epoch 6/100
3516/3516 [==============================] - 375s 107ms/step - loss: 0.1975 - accuracy: 0.9163 - val_loss: 0.4962 - val_accuracy: 0.8493 - lr: 0.0010
Epoch 7/100
3516/3516 [====

In [ ]:
del pos_model
for i in range(100):
    gc.collect()
    K.clear_session()
    tf.keras.backend.clear_session()
    torch.cuda.empty_cache()

In [19]:
test_input = np.load("./dataset3/testset/input1k_1.npy")
test_output = np.load("./dataset3/testset/output1k_1_num_class.npy")

#rpt_model=tf.keras.models.load_model("./best_model_effv2L_0413.h5")
rpt_model = tf.keras.models.load_model ('./weights_efficientv2.h5')
pred=rpt_model.predict(test_input)
true = 0
false = 0
pred_out = np.array([])
for i in range(pred.shape[0]):
    pred1=np.argmax(pred[i])
    pred_out=np.append(pred_out,pred1)
    if pred1==np.argmax(test_output[i]):
        true += 1
    else:
#        print(i)
        false +=1
acc = 100 * (true/(true+false))
print(true)
print(false)
print(acc,"%")

ResourceExhaustedError: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[640] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Fill]

In [58]:
pred_out[:40]

NameError: name 'pred_out' is not defined

(15, 32, 64, 3)


In [67]:
pred2=rpt_model.predict(test2)
np.argmax(pred2,axis=1)

1/1 [==============================] - 0s 24ms/step


array([2, 5, 5, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2], dtype=int64)

In [8]:
def find_start_end_mid(active_routing):
    a1=np.where(active_routing==1)[1]
    unique_elements, counts=np.unique(a1, return_counts=True)
    mid=unique_elements[counts>1][0]
    
    y1=np.where(active_routing==2)[0][1]
    y2=np.where(active_routing==2)[0][0]
    x1=np.where(active_routing==2)[1][1]
    x2=np.where(active_routing==2)[1][0]
    start= np.array([[y1,x1]])
    end = np.array([[y2,x2]])
    mid1 = np.array([[y1,mid]])
    mid2 = np.array([[y2,mid]])
    return start,end,mid1,mid2

In [34]:
def find_div_pos(divide,start,end,mid1,mid2):
    div_pos=np.copy(start)
    unit_space = np.array([unit_space_y, unit_space_x])
    line1_length = np.sum(np.multiply(abs(start[0]-mid1[0]), unit_space))
    line2_length = np.sum(np.multiply(abs(mid1[0]-mid2[0]), unit_space))
    line3_length = np.sum(np.multiply(abs(mid2[0]-end[0]), unit_space))
#    print(line1_length,line2_length,line3_length)
    distance = line1_length+line2_length+line3_length
    line_num=np.array([])
    for i in range(divide.shape[0]):
            
            if divide[i]<=line1_length :
                temp2 = np.copy(start)
                temp2[0][1] += np.round(divide[i]/unit_space_x)
                div_pos = np.append(div_pos,temp2,axis=0)
                line_num=np.append(line_num,np.array([1]))
            if line1_length<divide[i]<=line1_length+line2_length:
                temp2 = np.copy(mid1)
                temp2[0][0] -= np.round((divide[i]-line1_length)/unit_space_y)
                div_pos = np.append(div_pos,temp2,axis=0)
                line_num=np.append(line_num,np.array([2]))

            if divide[i] > line1_length+line2_length:
                temp2 = np.copy(mid2)
                temp2[0][1] += np.round((divide[i]-line1_length-line2_length)/unit_space_x)
                div_pos = np.append(div_pos,temp2,axis=0)
                line_num=np.append(line_num,np.array([3]))

                
    div_pos = np.delete(div_pos,0,axis=0)
    return div_pos,line_num,distance

In [35]:
def divide_line(active,min_rpt,start,end,mid1,mid2,add_num=0):
#    unit_space_x = int(target_x/active.shape[1])
#    unit_space_y = int(target_y/active.shape[0])
#    print("distance:",distance)
#    print("minimum # of repeaters:",min_rpt)
#    print("input:",x1,y1)
#    print("output:",x2,y2)
#    print("mid:",mid)
    min_rpt = min_rpt + add_num
    distance = abs(start-end)[0][0]*unit_space_y+abs(start-end)[0][1]*unit_space_x
    unit_distance = round(distance / (min_rpt+1) )
    divide = np.array([])
    divide_line = np.array([])
    for i in range(min_rpt):
        divide = np.round(np.append(divide, unit_distance*(i+1)))
        
    div_pos,line_num,distance = find_div_pos(divide,start,end,mid1,mid2)
    
    space=np.min(np.append(divide,distance)-np.append(0,divide))
    bound= int((target_length-space)/unit_space_x)        
    online_min=bound
    for i in range(div_pos.shape[0]-1):
        dist=int(math.dist(div_pos[i],div_pos[i+1]))
        if dist<online_min:
            online_min=dist
    return divide,div_pos,line_num,online_min,distance

In [36]:
def select_area(active,active_inout,divide,div_point,online_min):

    space=np.min(np.append(divide,distance)-np.append(0,divide))
    bound= int((target_length-space)/unit_space_x)

#    print(bound)
    x1=div_point[1]
    y1=div_point[0]
    a = np.array([[0,0]])
    
    if x1-bound < 0:
        
        b=np.arange(0,x1+bound+1)
    if (x1+bound) > (active.shape[1]-1):
        b=np.arange(x1-bound,active.shape[1])
    if x1-bound >= 0 and x1+bound <= active.shape[1]-1:
        b=np.arange(x1-bound,x1+bound+1)
        
    if y1-bound < 0:
        c=np.arange(0,y1+bound+1)
    if (y1+bound) > (active.shape[0]-1):
        c=np.arange(y1-bound,active.shape[0])
    if y1-bound >= 0 and y1+bound <= active.shape[0]-1:
        c=np.arange(y1-bound,y1+bound+1)   
    for i in range(c.shape[0]):
        for j in range(b.shape[0]):
            a=np.append(a,np.array([[c[i],b[j]]]),axis=0)
            
    a = np.delete(a,0,axis=0)
    c= np.array([[0, 0]])
    online=np.array([])
    for i in range(a.shape[0]):
        x2=a[i][1]
        y2=a[i][0]
        
        
        if active[y2][x2] == 0 and active_inout[y2][x2]==0:
            c=np.append(c,a[i].reshape(1,2),axis=0)
            online=np.append(online,0)
        if active[y2][x2] == 1 and active_inout[y2][x2]==0:
            if math.dist(div_point,a[i])<=online_min:
                c=np.append(c,a[i].reshape(1,2),axis=0)
                online=np.append(online,1)
            
    select= np.delete(c,0,axis=0)

    
    return select,online

In [37]:
def find_opt_delay(active,active_inout,divide,div_pos,online_min):
    rpt_dist=np.array([])
    opt_pos=np.array([[0,0]])
    pos=np.copy(div_pos)
    min_dist = 10000
    error=0
    for i in range(div_pos.shape[0]):
        error=0
        select,online = select_area(active,active_inout,divide,div_pos[i],online_min)
        if select.shape[0]==0:
            error=1
            print("#######################NO ACTIVE################")
            print(i)
#            break
        else:
            temp=abs(select-div_pos[i])
            min_d=np.argmin(np.sum(temp,axis=1))
            temp=np.sum(temp[min_d])
            if online[min_d]==1:
                pos[i]=select[min_d]
                temp=0
            opt_pos=np.append(opt_pos,select[min_d].reshape(1,2),axis=0)
            rpt_dist=np.append(rpt_dist,temp)
#            print(rpt_dist)
    opt_pos=np.delete(opt_pos,0,axis=0)
        
    
    return opt_pos,pos,rpt_dist,error

In [53]:
def cal_rpt_dist(active,opt_pos,pos,start,end,rpt_dist):

    temp = np.append(pos,end,axis=0)
    temp2 = np.append(start,pos,axis=0)
    part_dist = np.sum(abs(temp-temp2),axis=1)
    temp3 = np.append(rpt_dist,np.array(0))

    temp4 = np.append(np.array(0),rpt_dist)
    dist = (part_dist+temp3+temp4)*unit_space_x
    return dist

In [73]:
test2=np.copy(test_input[25:40])
test2_out=np.copy(test_output[25:40])
print(test2.shape)
#test2[test2==-1]=0

(15, 32, 64, 3)


In [74]:

for i in range(test2.shape[0]):
    
#    active=test_input[25,:,:,0]
#    active_inout=test_input[25,:,:,1]
#    active_routing=test_input[25,:,:,2]
    active=test2[i,:,:,0]
    active_inout=test2[i,:,:,1]
    active_routing=test2[i,:,:,2]
    target_length = 100
    unit_space_x = 5
    unit_space_y = 5

    add_num = 0

    start,end,mid1,mid2=find_start_end_mid(active_routing)
#    print("start,end:",start,end,mid1,mid2)
    distance = np.sum(np.multiply(abs(start-end),[unit_space_x,unit_space_y]))
#    print(distance)
    min_rpt = int(distance/target_length)
#    print("min_rpt",min_rpt)
    divide, div_pos,line_num,online_min,distance= divide_line(active,min_rpt,start,end,mid1,mid2,add_num)
#    print(divide)
#    print(div_pos)
#    print(line_num)
#    print(online_min)
#    print(distance)
    opt_pos,pos,rpt_dist,error=find_opt_delay(active,active_inout,divide,div_pos,online_min)
#    print(opt_pos)
#    print(error)

    print(rpt_dist)

    dist=cal_rpt_dist(active,opt_pos,pos,start,end,rpt_dist)
    print(dist)

[1. 1. 0.]
[95. 95. 95. 85.]
[1. 0. 3. 0.]
[ 85.  85. 100.  95.  80.]
[0. 1. 0. 0.]
[75. 95. 85. 80. 85.]
[2. 2. 0.]
[ 90. 100.  95.  80.]
[0. 0. 0.]
[80. 80. 80. 85.]
[2. 1. 2.]
[ 90. 100.  95.  95.]
[2. 0. 1.]
[105. 105.  95. 100.]
[1. 0. 0.]
[85. 90. 75. 80.]
#######################NO ACTIVE################
2
[0. 0.]


ValueError: operands could not be broadcast together with shapes (4,) (3,) 

In [76]:
np.argmax(test2_out,axis=1)+1

array([3, 6, 6, 3, 4, 3, 4, 3, 4, 4, 3, 3, 3, 3, 4], dtype=int64)

In [ ]:
############################POS_PREDICT###############################################################################################################

In [ ]:
del pos_model
for i in range(200):
    if i%50==0:
        time.sleep(2)
        print("2s")
    gc.collect()
    K.clear_session()
    tf.keras.backend.clear_session()
    torch.cuda.empty_cache()

In [5]:
r_input_pos = np.load("./gnn/dataset3/input500k_pos.npy")[:100000]
r_output_pos = np.load("./gnn/dataset3/output500k_pos.npy")[:100000]
r_output=np.load("./gnn/dataset3/output500k.npy")[:100000]
print(r_input_pos.shape)
print(r_output_pos.shape)
print(r_output.shape)

(100000, 32, 64, 3)
(100000, 32, 64)
(100000, 32, 64)


In [6]:
new_input_1=np.copy(r_input_pos[:,:,:,0])
new_input_2=np.copy(r_input_pos[:,:,:,1])
new_input_3=np.copy(r_input_pos[:,:,:,2])
new_input_1[new_input_1 !=4] = 0
new_input_2[new_input_2 ==2] = -1
new_input_3[new_input_3 !=2] = 0
r_input_pos[:,:,:,0]=new_input_1
r_input_pos[:,:,:,1]=new_input_2
r_input_pos[:,:,:,2]=new_input_3
#print(new_input_2[0])

In [7]:
print(r_input_pos[0,:,:,0])
print(r_input_pos[0,:,:,1])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [21]:
r_output_flat = r_output.reshape(100000,-1)

In [20]:
num_filters = 320

eff_net=tf.keras.applications.EfficientNetV2S(weights="imagenet",input_shape=(32,64,3),include_top=False)

pos_model = tf.keras.Sequential([
    eff_net,
    Conv2D(num_filters, kernel_size=(3,3), activation='relu', padding='same', input_shape = (32,64,3)),
    UpSampling2D((2,2)),
    Conv2D(num_filters/2, kernel_size=(3,3), activation='relu', padding='same', input_shape = (32,64,3)),
    UpSampling2D((2,2)),
    Conv2D(num_filters/4, kernel_size=(3,3), activation='relu', padding='same', input_shape = (32,64,3)),
    UpSampling2D((2,2)),
    Conv2D(num_filters/8, kernel_size=(3,3), activation='relu', padding='same', input_shape = (32,64,3)),
    UpSampling2D((2,2)),
    Conv2D(num_filters/16, kernel_size=(3,3), activation='relu', padding='same', input_shape = (32,64,3)),
    UpSampling2D((2,2)),
    Conv2D(filters=1, kernel_size=1, activation='sigmoid'),
    Reshape((32,64)),
    Flatten()
    
#    Conv2D(num_filters, kernel_size=(3,3), activation='relu', padding='same', input_shape = (32,64,3)),
#    MaxPool2D((2,2), padding='same'),
#    Conv2D(num_filters*2, kernel_size=(3,3), activation='relu', padding='same'),
#    MaxPool2D((2,2), padding='same'),
#    Conv2D(num_filters*4, kernel_size=(3,3), activation='relu', padding='same'),
#    UpSampling2D((2,2)),
#    Conv2D(num_filters*2, kernel_size=(3,3), activation='relu', padding='same'),
#    UpSampling2D((2,2)),
#    Conv2D(1, kernel_size=(3,3), activation='sigmoid', padding='same'),
#    Reshape((32,64)),
#    Flatten()
    
    #Flatten(),
    #Dense(2048, activation='sigmoid')
    
    #Activation('softmax')
    #Dropout(0.5),
    #Dense(2048, activation='softmax')

])

pos_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Functiona  (None, 1, 2, 1280)       20331360  
 l)                                                              
                                                                 
 conv2d (Conv2D)             (None, 1, 2, 320)         3686720   
                                                                 
 up_sampling2d (UpSampling2D  (None, 2, 4, 320)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 4, 160)         460960    
                                                                 
 up_sampling2d_1 (UpSampling  (None, 4, 8, 160)        0         
 2D)                                                             
                                                        

In [ ]:
from sklearn.utils import class_weight


class_weights=class_weight.compute_class_weight(class_weight = 'balanced',classes = np.unique(r_output[0]), y=r_output.flatten())
print(class_weights)

class_weights2 = class_weights[1]
print(class_weights2)

class_weights3= class_weights2 - 290
print(class_weights3)

In [22]:
from sklearn.utils import class_weight

def weighted_binary_crossentropy(y_true, y_pred):
    epsilon = 1e-7
    
    y_true = tf.reshape(tf.convert_to_tensor(y_true), [-1])
    y_pred = tf.reshape(tf.convert_to_tensor(y_pred), [-1])
    y_true_np = y_true.numpy().astype('int32')
    y_true_float=tf.cast(y_true, tf.float64)
    y_pred_float=tf.cast(y_pred, tf.float64)
    class_weights = class_weight.compute_class_weight( class_weight = 'balanced', classes = [0, 1], y=y_true_np)
    y_pred_float = tf.clip_by_value(y_pred_float,epsilon,1-epsilon)
    loss = -y_true_float*tf.math.log(y_pred_float)*class_weights[1]-(1-y_true_float)*tf.math.log(1-y_pred_float)*class_weights[0]
    
    
#    bce = tf.keras.losses.BinaryCrossentropy()
#    loss = bce(y_true_float,y_pred_float, sample_weight=class_weights[1])
    return tf.reduce_mean(loss)



In [23]:

pos_model.compile(optimizer='Adam', loss=weighted_binary_crossentropy, metrics=['accuracy'],run_eagerly=True)
#pos_model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
pos_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Functiona  (None, 1, 2, 1280)       20331360  
 l)                                                              
                                                                 
 conv2d (Conv2D)             (None, 1, 2, 320)         3686720   
                                                                 
 up_sampling2d (UpSampling2D  (None, 2, 4, 320)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 4, 160)         460960    
                                                                 
 up_sampling2d_1 (UpSampling  (None, 4, 8, 160)        0         
 2D)                                                             
                                                        

In [24]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
save = tf.keras.callbacks.ModelCheckpoint('best_model_effv2_pos.h5', monitor='val_loss', mode='min', save_best_only=True)
batchs = 64
def scheduler(epoch):
    if epoch < 20:
        return 0.001
    else:
        return 0.001*math.exp(0.1*(10-epoch))

schedule = tf.keras.callbacks.LearningRateScheduler(scheduler)



callback = [es, save, schedule]

pos_model.fit(r_input_pos,r_output_flat, epochs=30, batch_size = batchs, validation_split =0.1, callbacks=callback,
             )
pos_model.save("./weights_efficientv2_pos.h5")

#class_weight={i:class_weights[i] for i in range(len(class_weights))}

Epoch 1/30
1407/1407 [==============================] - 630s 427ms/step - loss: 0.2606 - accuracy: 0.0103 - val_loss: 0.1752 - val_accuracy: 0.0168 - lr: 0.0010
Epoch 2/30
1407/1407 [==============================] - 599s 426ms/step - loss: 0.1543 - accuracy: 0.0220 - val_loss: 0.1829 - val_accuracy: 0.0110 - lr: 0.0010
Epoch 3/30
1407/1407 [==============================] - 600s 426ms/step - loss: 0.1268 - accuracy: 0.0291 - val_loss: 0.1263 - val_accuracy: 0.0266 - lr: 0.0010
Epoch 4/30
1407/1407 [==============================] - 604s 430ms/step - loss: 0.1165 - accuracy: 0.0332 - val_loss: 0.1110 - val_accuracy: 0.0351 - lr: 0.0010
Epoch 5/30
1407/1407 [==============================] - 597s 425ms/step - loss: 0.1114 - accuracy: 0.0351 - val_loss: 0.1172 - val_accuracy: 0.0388 - lr: 0.0010
Epoch 6/30
1407/1407 [==============================] - 597s 425ms/step - loss: 0.1056 - accuracy: 0.0368 - val_loss: 0.1050 - val_accuracy: 0.0547 - lr: 0.0010
Epoch 7/30
1407/1407 [============

KeyboardInterrupt: 

In [ ]:
#np.where(pos_model.predict(r_input_pos[:100])>0)
custom_objects = {"weighted_binary_crossentropy": weighted_binary_crossentropy}
pos_model = tf.keras.models.load_model ('./weights_efficientv2_pos.h5',custom_objects=custom_objects)


test=pos_model.predict(r_input_pos[:2])
test=np.where(test[0]>=0.99,1,0)

In [ ]:
print(test.reshape(32,64))

In [ ]:
print(r_output[0])

In [ ]:
pos_model=tf.keras.models.load_model("./best_model_effv2_pos.h5")
#model3 = tf.keras.models.load_model ('./weights_efficientv2.h5')
pred=pos_model.predict(test_input)
pred.shape
true = 0
false = 0
for i in range(pred.shape[0]):
    if np.argmax(pred[i])==np.argmax(test_output[i]):
        true += 1
    else:
        print(i)
        false +=1
acc = 100 * (true/(true+false))
print(true)
print(false)
print(acc,"%")

In [ ]:
threshold = 0.000000001
print(r_input_pos[0].shape)
pred=pos_model.predict(r_input_pos[0].reshape(1,32,64,3))

In [ ]:
print(np.array(np.where(pred > threshold, 1,0)).reshape(32,64))